In [17]:
%matplotlib notebook
# %matplotlib inline 
# %matplotlib qt
import numpy as np
import cv2
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import glob
import xlsxwriter
import random

# Функции

In [32]:
# ищет самые крайние точки
def get_borders_last(image):
    res_non_zero_vert = np.argwhere((image > 1).any(axis=1))
    res_non_zero_horz = np.argwhere((image[res_non_zero_vert[0][0]] > 1))[-1]

    A = (res_non_zero_vert[0][0],  np.argwhere((image[res_non_zero_vert[0][0]] > 1))[-1][0])
    B = (res_non_zero_vert[-1][0], np.argwhere((image[res_non_zero_vert[-1][0]] > 1))[-1][0])

    return np.array([A, B])


# ищет точки как центр масс вокруг самых крайних
def get_borders_cm(image, sigma=10):
    A, B = get_borders_last(image)

    A_neighborhood = image[(A[0]-sigma):(A[0]+sigma), (A[1]-sigma):(A[1]+sigma)]
    B_neighborhood = image[(B[0]-sigma):(B[0]+sigma), (B[1]-sigma):(B[1]+sigma)]
    
    new_A = A - sigma + get_center_manual(A_neighborhood)
    new_B = B - sigma + get_center_manual(B_neighborhood)
    return new_A, new_B

    
    
def get_center_contour(image):
    ret,thresh = cv2.threshold(image.astype('uint8'),100,255,0)
    contours, hierachy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]

    M = cv2.moments(cnt)
    
    print(M)

    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    return cx, cy

def get_center_manual(image):
    [x, y] = np.mgrid[0:image.shape[0], 0:image.shape[1]]    ;
    weightedx = x * image;
    weightedy = y * image;
    xcentre = np.sum(weightedx) / np.sum(image);
    ycentre = np.sum(weightedy) / np.sum(image);
    return np.array([xcentre, ycentre])

def load_images(path):
    filenames = [img for img in glob.glob(path)] 
    filenames.sort()

    images = []
    for img in filenames:
        n= cv2.imread(img)
        n = cv2.cvtColor(n, cv2.COLOR_BGR2GRAY)
        n = n.astype('float')
        images.append(n)
    return images

# сглаживает изображение оконным фильтром и фильтрует по порогу
def filter_images(images, filter_window_size = 10, threshold = 50):
    filtered_images = original_images.copy()
    for i, img in zip(range(len(images)), images):
        filtered_image = img.copy()

        filtered_image = filtered_image[:, :1100]#int(filtered_image.shape[1] / 2)]

        kernel = np.ones((filter_window_size,filter_window_size), np.float32) / (filter_window_size * filter_window_size)

        filtered_image = cv2.filter2D(filtered_image, -1, kernel)

        np.putmask(filtered_image, filtered_image > threshold, 255)
        np.putmask(filtered_image, filtered_image <= threshold, 0)
#       np.putmask(filtered_image, filtered_image < threshold, 0)

        filtered_images[i] = filtered_image
    return filtered_images

# поиск крайних точек, поиск ближайшего контура, поиск срднего в контуре
def get_borders_cm_contour(image, sigma=10, threshold=50):
    A, B = get_borders_last(image)

    A_neighborhood = image[(A[0]-sigma):(A[0]+sigma), (A[1]-sigma):(A[1]+sigma)]
    B_neighborhood = image[(B[0]-sigma):(B[0]+sigma), (B[1]-sigma):(B[1]+sigma)]
    
    _A = get_center_cm_contour(A_neighborhood, threshold)
    _B = get_center_cm_contour(B_neighborhood, threshold)
    
            
#     plt.figure("Show neighborhood A")
#     plt.imshow(A_neighborhood, cmap='gray', vmin=0, vmax=255)
#     plt.plot(_A[0], _A[1], 'ro', markersize=3)
    
#     plt.figure("Show neighborhood B")
#     plt.imshow(B_neighborhood, cmap='gray', vmin=0, vmax=255)
#     plt.plot(_B[0], _B[1], 'ro', markersize=3)
    
    
#     print("A" + str(A))
    new_A = A - sigma + (_A[1], _A[0])
#     print("new_A" + str(new_A))
    new_B = B - sigma + (_B[1], _B[0])
    return new_A, new_B


# поиск контура и его среднего
def get_center_cm_contour(image_cm, threshold):
    ret,thresh = cv2.threshold(image_cm.astype('uint8'), threshold, 255,0) # первый параметр - нижний порог threshold
    contours, hierachy = cv2.findContours(thresh, 1, 2)
    
#     print("found " + str(len(contours)) + " contours.")
    
    dist_max = -10 * image_cm.shape[0]
    
    cx = 1
    cy = 1
    
    for cnt in contours:
        pt = (int(image_cm.shape[0] / 2) - 1,int(image_cm.shape[0] / 2) - 1)
        dist = cv2.pointPolygonTest(cnt, pt,True)
#         print(dist)
        if dist >= dist_max:
            dist_max = dist
            M = cv2.moments(cnt, True)
            
            cx = int(M['m10'] / (M['m00']  + 1e-5) )
            cy = int(M['m01'] / (M['m00']  + 1e-5))
            
#             if M['m00'] != 0:
#                 cx = int(M['m10'] / (M['m00']  + 1e-5) )
#                 cy = int(M['m01'] / (M['m00']  + 1e-5))
#             else:
#                 # set values as what you need in the situation
#                 cx, cy = 0, 0
#                 print(M)
#                 print(len(contours))
#                 print(threshold)
#                 plt.figure("Show neighborhood in error place")
#                 # image = cv2.drawContours(image_cm, cnt, -1, (255,0,0), 1)
#                 image = image_cm
#                 plt.imshow(image, cmap='gray', vmin=0, vmax=255)
#                 plt.savefig(str(random.randint(0, 100)) + 'result.png', dpi=800)
    return cx, cy

def calculate_sko(images, threshold, sigma):
    filtered_images = filter_images(original_images, threshold=threshold)

    A_arr = np.zeros(2 * images_number).reshape(images_number, 2)
    B_arr = np.zeros(2 * images_number).reshape(images_number, 2)

    ind = 0
    
    for i, img in zip(range(len(filtered_images)), filtered_images):

        x = get_borders_cm_contour(img, sigma = sigma, threshold=threshold)

        A_arr[i, 0] = x[0][0]
        A_arr[i, 1] = x[0][1]
        B_arr[i, 0] = x[1][0]
        B_arr[i, 1] = x[1][1]
        
        # draw
        cv2.circle(img, (int(A_arr[i, 1]), int(A_arr[i, 0])), 5, (255, 0, 0))
        cv2.circle(img, (int(B_arr[i, 1]), int(B_arr[i, 0])), 5, (255, 0, 0))
        
        # save
        name = 'Test_gray' + str(ind) + '.bmp'
        cv2.imwrite('photos_with_borders/' + name, img) 
        ind += 1
        
        
#     coefficient = 0.00760673
#     true_diametr = 8.1823

    dy = B_arr[:,0] - A_arr[:,0]
    
    st = np.stack((A_arr[:,1], B_arr[:,1]))
    mean_x_px = np.mean((B_arr[:,1], A_arr[:,1]), axis = 0)
    
    # функция получения срденего по х в мм =0,000000577*х_пикс*х_пикс+0,0102407159*х_пикс+65,7471592014,
    # где х_пикс - среднее от А и B по Х
    mean_x_mm = 0.000000577 * np.power(mean_x_px, 2) + 0.0102407159 * mean_x_px + 65.7471592014
    
    # функция калибровки coefficient = 5/(0,045*x*x-11,55*x+1274), x - средняя от А и B координата в мм
    coefficients = 5/(0.045 * np.power(mean_x_mm, 2) - 11.55 * mean_x_mm + 1274)
    
    diametr = coefficients * dy
    std = np.std(diametr) * 1000

    return std

## Load images

In [12]:
original_images = load_images("photo/different_diametrs/8_1823/*.bmp")
images_number = len(original_images)

## Calculate SKO

In [34]:
from_t = 40
to_t = 50
step_t = 5

from_s = 5
to_s = 20 # 30
step_s = 2

number_t = int((to_t - from_t)/step_t)
number_s = int((to_s - from_s)/step_s)

sko = np.zeros((number_t, number_s))

for i, threshold in zip(range(number_t), range(from_t, to_t, step_t)):
    for j, sigma in zip(range(number_s), range(from_s, to_s, step_s)):
        print("sigma" + str(sigma))
        sko[i, j] = calculate_sko(original_images, threshold=threshold, sigma = sigma)
        print(sko[i, j])

sigma5
128.47641454876128
sigma7
131.946483254237
sigma9
133.7015601926139
sigma11
137.2080574189652
sigma13
141.11914619235404
sigma15
145.22177283872966
sigma17
147.6833770851744
sigma5
123.19489906037137
sigma7
124.97211068028291
sigma9
127.04452995286624
sigma11
129.35742485439206
sigma13
132.60535589229542
sigma15
133.65856772836307
sigma17
133.58524742045682


## Writing data to EXCEL file

In [35]:
###### запись данных в файл
ar=np.asarray(sko)
# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('8_1823_.xlsx')
worksheet = workbook.add_worksheet()

# Add a bold format to use to highlight cells.
bold = workbook.add_format({'bold': 1})

# Add a number format for cells with data.
header_format = workbook.add_format({'num_format': '000010'})

# Adjust the column width.
worksheet.set_column(0, 0, 15)

# Write some headers
worksheet.write('A1', 'threshold/sigma', bold)

# Create heders of range sigma 
col = 0
for sigma in range(from_s, to_s, step_s):
    worksheet.write_number  (0, col +1, sigma)
    col += 1

# Create heders of range threshold
row = 0
for threshold in range(from_t, to_t, step_t):
    worksheet.write_number  (row +1, 0 , threshold)
    row += 1

# Write data

for row_num, row_data in enumerate(ar):
    for col_num, col_data in enumerate(row_data):
        worksheet.write(row_num + 1, col_num + 1, col_data)

    
workbook.close()

print (np.amin(ar))
print('done')


123.19489906037137
done
